In [ ]:
!unzip 'drive/My Drive/ViolentFlows.zip'

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-bebd7d95-017f-fcf8-5df7-0ea414c12aa2)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 #Architecture 7 with disk size 9
import numpy as np
import cv2
import glob
from scipy import io
import matplotlib.pyplot as plt
import random
from keras.utils import to_categorical

#from keras.utils import np_utils
import tensorflow as tf
from skimage.filters.rank import entropy
from skimage.morphology import disk
VideoPath = "ViolentFlows/Violence/"
Videos = glob.glob(VideoPath + "*.avi")
#uint8
XTrainImages=np.zeros((246,50,80,16),dtype='float16')
Counter=0
for vid in Videos:
    X = []
    cap=cv2.VideoCapture(vid)
    ret, frame = cap.read()
    FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    OldFrame=cv2.resize(FrameGray,(80,50))
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            NewFrame=cv2.resize(FrameGray,(80,50))
            EntropyFilter=entropy(abs(NewFrame-OldFrame), disk(9))
            X.append(abs(NewFrame-OldFrame)*EntropyFilter+OldFrame)
            OldFrame=NewFrame
            #plt.imshow(FrameGray,cmap=plt.cm.gray)
        else:
            break
    tt=random.sample(X,16)
    for i in range(len(tt)):
        XTrainImages[Counter,:,:,i]=tt[i]
    Counter=Counter+1

VideoPath = "ViolentFlows/NonViolence/"
Videos = glob.glob(VideoPath + "*.avi")
for vid in Videos:
    X = []
    cap=cv2.VideoCapture(vid)
    ret, frame = cap.read()
    FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    OldFrame=cv2.resize(FrameGray,(80,50))
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            NewFrame=cv2.resize(FrameGray,(80,50))
            EntropyFilter=entropy(abs(NewFrame-OldFrame), disk(9))

            X.append(abs(NewFrame-OldFrame)*EntropyFilter+OldFrame)
            OldFrame=NewFrame
            #plt.imshow(FrameGray,cmap=plt.cm.gray)
        else:
            break
    tt=random.sample(X,16)
    for i in range(len(tt)):
        XTrainImages[Counter,:,:,i]=tt[i]
    Counter=Counter+1
#XTrainImages/=255
## Deep Network



In [ ]:
# Violent Flows Disk size=9 Global Average pooling

from keras import layers
from keras.models import Model
from keras import optimizers, losses
import numpy as np
from keras.utils import to_categorical
from keras import optimizers
from keras.losses import categorical_crossentropy
TrainLabels=np.zeros((246,1),dtype='int8')
TrainLabels[123:]=1
TrainLabels=to_categorical(TrainLabels)
x_train = np.reshape(XTrainImages, (len(XTrainImages), 50, 80, 16,1))
input_img = layers.Input(shape=(50, 80, 16,1))  # adapt this if using `channels_first` image data format
#x = layers.Conv3D(1, (5,5,5), activation='relu', padding='same')(input_img)
#x=layers.Reshape((50, 80, 25))(x)
x=layers.Reshape((50, 80, 16))(input_img)
y=layers.GlobalAveragePooling2D()(x)
#y=layers.MaxPool2D(pool_size=(5,8),strides=(10,10))(x)
#y=layers.Flatten()(y)

y = layers.Dense(2, activation='relu')(y)
y=layers.Dropout(0.3)(y)
y = layers.Dense(16, activation='sigmoid')(y)
y=layers.Dropout(0.3)(y)
z = tf.keras.layers.Multiply()([y, x])
x = layers.Conv2D(3, (1,1), activation='elu', padding='same')(z)

# z=layers.Reshape((50, 80, 16,1))(z)
# x = layers.Conv3D(1, (5,5,14), activation='elu', padding='valid')(z)
# x=layers.Reshape((46, 76, 3))(x)

x = layers.Conv2D(10, (5,5), activation='elu', padding='same')(x)
x=layers.MaxPool2D(pool_size=(2,2))(x)
x = layers.BatchNormalization()(x)

y=layers.Flatten()(x)
y = layers.Dense(300, activation='elu')(y)
y=layers.Dropout(0.2)(y)

y = layers.Dense(100, activation='elu')(y)
y=layers.Dropout(0.2)(y)

x = layers.Conv2D(20, (3,3), activation='elu', padding='same')(x)
x=layers.MaxPool2D(pool_size=(2,2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(30, (3,3), activation='elu', padding='same')(x)
x=layers.MaxPool2D(pool_size=(2,2))(x)
x = layers.BatchNormalization()(x)

x = layers.Flatten()(x)
x=layers.Concatenate()([x,y])
x = layers.Dense(300, activation='elu')(x)
x=layers.Dropout(0.2)(x)
x= layers.Dense(100, activation='elu')(x)
x=layers.Dropout(0.2)(x)
out_layer = layers.Dense(2, activation='softmax')(x)
myModel= Model(input_img, out_layer)
## Five Flod cross validation with RMSprop
from sklearn.model_selection import KFold
import numpy as np
import keras
myModel.compile()
myModel.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.optimizers.RMSprop(learning_rate=0.0001), metrics=['acc'])


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
myModel.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.optimizers.RMSprop(learning_rate=0.0001), metrics=['acc'])

# Assuming myModel is already defined

X = np.arange(len(TrainLabels))
Test_auc_roc = []
Test_precision = []
Test_recall = []
Test_f1 = []
All_confusion_matrices = []
Test_accuracy=[]
Total_TP=[]
Total_FP=[]
Total_FN=[]
Total_TN=[]
Counter = 0
for i in range(6):
    kf = KFold(n_splits=5, random_state=12345, shuffle=True)
    for train_indexInit, test_indexInit in kf.split(X):
        X_trainInit, X_test = x_train[train_indexInit], x_train[test_indexInit]
        y_trainInit, y_test = TrainLabels[train_indexInit], TrainLabels[test_indexInit]

        #print(test_indexInit)

        history = myModel.fit(X_trainInit, y_trainInit, batch_size=48, epochs=20, verbose=0)

        # Assuming myModel.predict is available for obtaining raw predictions
        y_pred = myModel.predict(X_test)
        test_loss, test_acc =myModel.evaluate(X_test, y_test)
        Test_accuracy.append(test_acc)
        print(Test_accuracy)
         # Confusion matrix
        cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
        All_confusion_matrices.append(cm)
        TP=cm[0,0]
        FP=cm[0,1]
        FN=cm[1,0]
        TN=cm[1,1]
        # Calculate ROC-AUC
        auc_roc = roc_auc_score(y_test, y_pred)
        Test_auc_roc.append(auc_roc)
        Total_TP.append(TP)
        Total_FP.append(FP)
        Total_FN.append(FN)
        Total_TN.append(TN)
        print(f"AUC-ROC: {auc_roc}, TP: {TP}, FP: {FP}, FN: {FN},TN:{TN}")
        print("Confusion Matrix:")
        print(cm)

        print(Counter)
        Counter += 1

print("Average AUC-ROC:", np.mean(Test_auc_roc))
print("Average Total_TP:", np.mean(Total_TP))
print("Average Total_FP:", np.mean(Total_FP))
print("Average Total_FN:", np.mean(Total_FN))
print("Average Total_TN:", np.mean(Total_TN))
Average_confusion_matrix = np.mean(All_confusion_matrices, axis=0)
print("Average Confusion Matrix:")
print(Average_confusion_matrix)
print("Test_accuracy",Test_accuracy)
print("Average Test Accurcay",np.mean(Test_accuracy))
print("SD Test Accuracy",np.std(Test_accuracy))
precision=np.mean(Total_TP)/(np.mean(Total_TP)+np.mean(Total_FP))
recall=np.mean(Total_TP)/(np.mean(Total_TP)+np.mean(Total_FN))
f1score=2*precision*recall/(precision+recall)
print("Average precision:", precision)
print("Average recall:",recall)
print("Average f1score:", f1score)



2/2 [==============================] - 0s 15ms/step - loss: 1.0404 - acc: 0.6200
[0.6200000047683716]
AUC-ROC: 0.6784, TP: 11, FP: 14, FN: 5,TN:20
Confusion Matrix:
[[11 14]
 [ 5 20]]
0
2/2 [==============================] - 0s 12ms/step - loss: 0.1282 - acc: 0.9592
[0.6200000047683716, 0.9591836929321289]
AUC-ROC: 0.9949832775919734, TP: 23, FP: 0, FN: 2,TN:24
Confusion Matrix:
[[23  0]
 [ 2 24]]
1
2/2 [==============================] - 0s 9ms/step - loss: 0.0080 - acc: 1.0000
[0.6200000047683716, 0.9591836929321289, 1.0]
AUC-ROC: 1.0, TP: 25, FP: 0, FN: 0,TN:24
Confusion Matrix:
[[25  0]
 [ 0 24]]
2
2/2 [==============================] - 0s 11ms/step - loss: 0.0040 - acc: 1.0000
[0.6200000047683716, 0.9591836929321289, 1.0, 1.0]
AUC-ROC: 1.0, TP: 26, FP: 0, FN: 0,TN:23
Confusion Matrix:
[[26  0]
 [ 0 23]]
3
2/2 [==============================] - 0s 14ms/step - loss: 3.0940e-04 - acc: 1.0000
[0.6200000047683716, 0.9591836929321289, 1.0, 1.0, 1.0]
AUC-ROC: 1.0, TP: 24, FP: 0, FN: 0,TN:

In [ ]:
!unzip 'drive/My Drive/SurvillanceDataset.zip'


In [ ]:
 # disk size 9 on Survillance dataset
import numpy as np
import cv2
import glob
from scipy import io
import matplotlib.pyplot as plt
import random
from keras.utils import to_categorical
import tensorflow as tf
from skimage.filters.rank import entropy
from skimage.morphology import disk
VideoPath = "SurvillanceDataset/fight/"
Videos = glob.glob(VideoPath + "*.mp4")
#uint8
XTrainImages=np.zeros((300,100,198,16),dtype='float16')
Counter=0
for vid in Videos:
    X = []
    cap=cv2.VideoCapture(vid)
    ret, frame = cap.read()
    FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    OldFrame=cv2.resize(FrameGray,(198,100))
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            NewFrame=cv2.resize(FrameGray,(198,100))
            EntropyFilter=entropy(abs(NewFrame-OldFrame), disk(9))
            X.append(abs(NewFrame-OldFrame)*EntropyFilter+OldFrame)
            OldFrame=NewFrame
            #plt.imshow(FrameGray,cmap=plt.cm.gray)
        else:
            break
    tt=random.sample(X,16)
    for i in range(len(tt)):
        XTrainImages[Counter,:,:,i]=tt[i]
    Counter=Counter+1

VideoPath = "SurvillanceDataset/noFight/"
Videos = glob.glob(VideoPath + "*.mp4")
for vid in Videos:
    X = []
    cap=cv2.VideoCapture(vid)
    ret, frame = cap.read()
    FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    OldFrame=cv2.resize(FrameGray,(198,100))
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            NewFrame=cv2.resize(FrameGray,(198,100))
            EntropyFilter=entropy(abs(NewFrame-OldFrame), disk(9))

            X.append(abs(NewFrame-OldFrame)*EntropyFilter+OldFrame)
            OldFrame=NewFrame
            #plt.imshow(FrameGray,cmap=plt.cm.gray)
        else:
            break
    tt=random.sample(X,16)
    for i in range(len(tt)):
        XTrainImages[Counter,:,:,i]=tt[i]
    Counter=Counter+1
#XTrainImages/=255
## Deep Network



In [ ]:
# Global Average Pooling, disk size=9, Survielnace Fight dataset
from keras import layers
from keras.models import Model
from keras import optimizers, losses
import numpy as np
from keras.utils import to_categorical
from keras import optimizers
from keras.losses import categorical_crossentropy
TrainLabels=np.zeros((300,1),dtype='int8')
TrainLabels[149:]=1
TrainLabels= to_categorical(TrainLabels)
x_train = np.reshape(XTrainImages, (len(XTrainImages), 100, 198, 16,1))
input_img = layers.Input(shape=(100, 198, 16,1))  # adapt this if using `channels_first` image data format
#x = layers.Conv3D(1, (5,5,5), activation='relu', padding='same')(input_img)
#x=layers.Reshape((50, 80, 25))(x)
x=layers.Reshape((100, 198, 16))(input_img)
y=layers.GlobalAveragePooling2D()(x)
#y=layers.MaxPool2D(pool_size=(5,8),strides=(10,10))(x)
#y=layers.Flatten()(y)

y = layers.Dense(2, activation='relu')(y)
y=layers.Dropout(0.3)(y)
y = layers.Dense(16, activation='sigmoid')(y)
y=layers.Dropout(0.3)(y)

z = tf.keras.layers.Multiply()([y, x])
x = layers.Conv2D(3, (1,1), activation='elu', padding='same')(z)

# z=layers.Reshape((100, 198, 16,1))(z)
# x = layers.Conv3D(1, (5,5,14), activation='elu', padding='valid')(z)
# x=layers.Reshape((96, 194, 3))(x)

x = layers.Conv2D(10, (5,5), activation='elu', padding='same')(x)
x=layers.MaxPool2D(pool_size=(2,2))(x)
x = layers.BatchNormalization()(x)

y=layers.Flatten()(x)
y = layers.Dense(300, activation='elu')(y)
y=layers.Dropout(0.2)(y)

y = layers.Dense(100, activation='elu')(y)
y=layers.Dropout(0.2)(y)

x = layers.Conv2D(20, (3,3), activation='elu', padding='same')(x)
x=layers.MaxPool2D(pool_size=(2,2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(30, (3,3), activation='elu', padding='same')(x)
x=layers.MaxPool2D(pool_size=(2,2))(x)
x = layers.BatchNormalization()(x)

x = layers.Flatten()(x)
x=layers.Concatenate()([x,y])
x = layers.Dense(300, activation='elu')(x)
x=layers.Dropout(0.2)(x)
x= layers.Dense(100, activation='elu')(x)
x=layers.Dropout(0.2)(x)
out_layer = layers.Dense(2, activation='softmax')(x)
myModel= Model(input_img, out_layer)
## Five Flod cross validation with RMSprop
from sklearn.model_selection import KFold
import numpy as np
import keras
myModel.compile()
myModel.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.optimizers.RMSprop(learning_rate=0.0001), metrics=['acc'])
import matplotlib.pyplot as plt


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
myModel.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.optimizers.RMSprop(learning_rate=0.0001), metrics=['acc'])

# Assuming myModel is already defined

X = np.arange(len(TrainLabels))
Test_auc_roc = []
Test_precision = []
Test_recall = []
Test_f1 = []
All_confusion_matrices = []
Test_accuracy=[]
Total_TP=[]
Total_FP=[]
Total_FN=[]
Total_TN=[]
Counter = 0
for i in range(6):
    kf = KFold(n_splits=5, random_state=12345, shuffle=True)
    for train_indexInit, test_indexInit in kf.split(X):
        X_trainInit, X_test = x_train[train_indexInit], x_train[test_indexInit]
        y_trainInit, y_test = TrainLabels[train_indexInit], TrainLabels[test_indexInit]

        #print(test_indexInit)

        history = myModel.fit(X_trainInit, y_trainInit, batch_size=48, epochs=20, verbose=0)

        # Assuming myModel.predict is available for obtaining raw predictions
        y_pred = myModel.predict(X_test)
        test_loss, test_acc =myModel.evaluate(X_test, y_test)
        Test_accuracy.append(test_acc)
        print(Test_accuracy)
         # Confusion matrix
        cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
        All_confusion_matrices.append(cm)
        TP=cm[0,0]
        FP=cm[0,1]
        FN=cm[1,0]
        TN=cm[1,1]
        # Calculate ROC-AUC
        auc_roc = roc_auc_score(y_test, y_pred)
        Test_auc_roc.append(auc_roc)
        Total_TP.append(TP)
        Total_FP.append(FP)
        Total_FN.append(FN)
        Total_TN.append(TN)
        print(f"AUC-ROC: {auc_roc}, TP: {TP}, FP: {FP}, FN: {FN},TN:{TN}")
        print("Confusion Matrix:")
        print(cm)

        print(Counter)
        Counter += 1

print("Average AUC-ROC:", np.mean(Test_auc_roc))
print("Average Total_TP:", np.mean(Total_TP))
print("Average Total_FP:", np.mean(Total_FP))
print("Average Total_FN:", np.mean(Total_FN))
print("Average Total_TN:", np.mean(Total_TN))
Average_confusion_matrix = np.mean(All_confusion_matrices, axis=0)
print("Average Confusion Matrix:")
print(Average_confusion_matrix)
print("Test_accuracy",Test_accuracy)
print("Average Test Accurcay",np.mean(Test_accuracy))
print("SD Test Accuracy",np.std(Test_accuracy))
precision=np.mean(Total_TP)/(np.mean(Total_TP)+np.mean(Total_FP))
recall=np.mean(Total_TP)/(np.mean(Total_TP)+np.mean(Total_FN))
f1score=2*precision*recall/(precision+recall)
print("Average precision:", precision)
print("Average recall:",recall)
print("Average f1score:", f1score)



2/2 [==============================] - 1s 28ms/step - loss: 1.2369 - acc: 0.6167
[0.6166666746139526]
AUC-ROC: 0.6764705882352942, TP: 21, FP: 13, FN: 10,TN:16
Confusion Matrix:
[[21 13]
 [10 16]]
0
2/2 [==============================] - 0s 23ms/step - loss: 0.1167 - acc: 0.9500
[0.6166666746139526, 0.949999988079071]
AUC-ROC: 1.0, TP: 19, FP: 3, FN: 0,TN:38
Confusion Matrix:
[[19  3]
 [ 0 38]]
1
2/2 [==============================] - 0s 21ms/step - loss: 0.0534 - acc: 0.9833
[0.6166666746139526, 0.949999988079071, 0.9833333492279053]
AUC-ROC: 1.0, TP: 33, FP: 0, FN: 1,TN:26
Confusion Matrix:
[[33  0]
 [ 1 26]]
2
2/2 [==============================] - 0s 24ms/step - loss: 0.0261 - acc: 0.9833
[0.6166666746139526, 0.949999988079071, 0.9833333492279053, 0.9833333492279053]
AUC-ROC: 1.0, TP: 23, FP: 1, FN: 0,TN:36
Confusion Matrix:
[[23  1]
 [ 0 36]]
3
2/2 [==============================] - 0s 22ms/step - loss: 0.0160 - acc: 1.0000
[0.6166666746139526, 0.949999988079071, 0.983333349227905

In [ ]:
!unzip 'drive/My Drive/Hockey Fight.zip'


In [ ]:
# Hockey Fight data set, disk size=9
import numpy as np
import cv2
import glob
from scipy import io
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from skimage.filters.rank import entropy
from skimage.morphology import disk
VideoPath = "Hockey Fight/Fi/"
Videos = glob.glob(VideoPath + "*.avi")
#uint8
XTrainImages=np.zeros((1000,60,90,16),dtype='float16')
Counter=0
for vid in Videos:
    X = []
    cap=cv2.VideoCapture(vid)
    ret, frame = cap.read()
    FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    OldFrame=cv2.resize(FrameGray,(90,60))
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            NewFrame=cv2.resize(FrameGray,(90,60))
            EntropyFilter=entropy(abs(NewFrame-OldFrame), disk(9))
            X.append(OldFrame*EntropyFilter+OldFrame)
            OldFrame=NewFrame
            #plt.imshow(FrameGray,cmap=plt.cm.gray)
        else:
            break
    tt=random.sample(X,16)
    for i in range(len(tt)):
        XTrainImages[Counter,:,:,i]=tt[i]
    Counter=Counter+1

VideoPath = "Hockey Fight/non-Fi/"
Videos = glob.glob(VideoPath + "*.avi")
for vid in Videos:
    X = []
    cap=cv2.VideoCapture(vid)
    ret, frame = cap.read()
    FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    OldFrame=cv2.resize(FrameGray,(90,60))
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            NewFrame=cv2.resize(FrameGray,(90,60))
            EntropyFilter=entropy(abs(NewFrame-OldFrame), disk(9))

            X.append(OldFrame*EntropyFilter+OldFrame)
            OldFrame=NewFrame
            #plt.imshow(FrameGray,cmap=plt.cm.gray)
        else:
            break
    tt=random.sample(X,16)
    for i in range(len(tt)):
        XTrainImages[Counter,:,:,i]=tt[i]
    Counter=Counter+1

#XTrainImages/=255
## Deep Network



In [ ]:
# Hoceky Fight, disk size=9
from keras import layers
from keras.models import Model
from keras import optimizers, losses
import numpy as np
from keras.utils import to_categorical
from keras import optimizers
from keras.losses import categorical_crossentropy
TrainLabels=np.zeros((1000,1),dtype='int8')
TrainLabels[500:]=1
TrainLabels= to_categorical(TrainLabels)
x_train = np.reshape(XTrainImages, (len(XTrainImages), 60, 90, 16,1))
input_img = layers.Input(shape=(60, 90, 16,1))  # adapt this if using `channels_first` image data format
#x = layers.Conv3D(1, (5,5,5), activation='relu', padding='same')(input_img)
#x=layers.Reshape((50, 80, 25))(x)
x=layers.Reshape((60, 90, 16))(input_img)
y=layers.GlobalAveragePooling2D()(x)

y = layers.Dense(2, activation='relu')(y)
y=layers.Dropout(0.2)(y)
y = layers.Dense(16, activation='sigmoid')(y)
y=layers.Dropout(0.2)(y)

z = tf.keras.layers.Multiply()([y, x])
x = layers.Conv2D(3, (1,1), activation='elu', padding='same')(z)

# z=layers.Reshape((60, 90, 16,1))(z)
# x = layers.Conv3D(1, (5,5,14), activation='elu', padding='valid')(z)
# x=layers.Reshape((56, 86, 3))(x)

x = layers.Conv2D(10, (5,5), activation='elu', padding='same')(x)
x=layers.MaxPool2D(pool_size=(2,2))(x)
x = layers.BatchNormalization()(x)

y=layers.Flatten()(x)
y = layers.Dense(300, activation='elu')(y)
y=layers.Dropout(0.2)(y)

y = layers.Dense(100, activation='elu')(y)
y=layers.Dropout(0.2)(y)

x = layers.Conv2D(20, (3,3), activation='elu', padding='same')(x)
x=layers.MaxPool2D(pool_size=(2,2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(30, (3,3), activation='elu', padding='same')(x)
x=layers.MaxPool2D(pool_size=(2,2))(x)
x = layers.BatchNormalization()(x)

x = layers.Flatten()(x)
x=layers.Concatenate()([x,y])
x = layers.Dense(300, activation='elu')(x)
x=layers.Dropout(0.2)(x)
x= layers.Dense(100, activation='elu')(x)
x=layers.Dropout(0.2)(x)
out_layer = layers.Dense(2, activation='softmax')(x)
myModel= Model(input_img, out_layer)
## Five Flod cross validation with RMSprop
from sklearn.model_selection import KFold
import numpy as np
import keras
myModel.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.optimizers.RMSprop(learning_rate=0.0001), metrics=['acc'])


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
myModel.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.optimizers.RMSprop(learning_rate=0.0001), metrics=['acc'])

# Assuming myModel is already defined

X = np.arange(len(TrainLabels))
Test_auc_roc = []
Test_precision = []
Test_recall = []
Test_f1 = []
All_confusion_matrices = []
Test_accuracy=[]
Total_TP=[]
Total_FP=[]
Total_FN=[]
Total_TN=[]
Counter = 0
for i in range(6):
    kf = KFold(n_splits=5, random_state=12345, shuffle=True)
    for train_indexInit, test_indexInit in kf.split(X):
        X_trainInit, X_test = x_train[train_indexInit], x_train[test_indexInit]
        y_trainInit, y_test = TrainLabels[train_indexInit], TrainLabels[test_indexInit]

        #print(test_indexInit)

        history = myModel.fit(X_trainInit, y_trainInit, batch_size=64, epochs=20, verbose=0)

        # Assuming myModel.predict is available for obtaining raw predictions
        y_pred = myModel.predict(X_test)
        test_loss, test_acc =myModel.evaluate(X_test, y_test)
        Test_accuracy.append(test_acc)
        print(Test_accuracy)
         # Confusion matrix
        cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
        All_confusion_matrices.append(cm)
        TP=cm[0,0]
        FP=cm[0,1]
        FN=cm[1,0]
        TN=cm[1,1]
        # Calculate ROC-AUC
        auc_roc = roc_auc_score(y_test, y_pred)
        Test_auc_roc.append(auc_roc)
        Total_TP.append(TP)
        Total_FP.append(FP)
        Total_FN.append(FN)
        Total_TN.append(TN)
        print(f"AUC-ROC: {auc_roc}, TP: {TP}, FP: {FP}, FN: {FN},TN:{TN}")
        print("Confusion Matrix:")
        print(cm)

        print(Counter)
        Counter += 1

print("Average AUC-ROC:", np.mean(Test_auc_roc))
print("Average Total_TP:", np.mean(Total_TP))
print("Average Total_FP:", np.mean(Total_FP))
print("Average Total_FN:", np.mean(Total_FN))
print("Average Total_TN:", np.mean(Total_TN))
Average_confusion_matrix = np.mean(All_confusion_matrices, axis=0)
print("Average Confusion Matrix:")
print(Average_confusion_matrix)
print("Test_accuracy",Test_accuracy)
print("Average Test Accurcay",np.mean(Test_accuracy))
print("SD Test Accuracy",np.std(Test_accuracy))
precision=np.mean(Total_TP)/(np.mean(Total_TP)+np.mean(Total_FP))
recall=np.mean(Total_TP)/(np.mean(Total_TP)+np.mean(Total_FN))
f1score=2*precision*recall/(precision+recall)
print("Average precision:", precision)
print("Average recall:",recall)
print("Average f1score:", f1score)



7/7 [==============================] - 1s 11ms/step - loss: 0.2600 - acc: 0.9200
[0.9200000166893005]
AUC-ROC: 0.9865879291362226, TP: 88, FP: 15, FN: 1,TN:96
Confusion Matrix:
[[88 15]
 [ 1 96]]
0
7/7 [==============================] - 0s 16ms/step - loss: 0.0438 - acc: 0.9900
[0.9200000166893005, 0.9900000095367432]
AUC-ROC: 0.9991815856777494, TP: 85, FP: 0, FN: 2,TN:113
Confusion Matrix:
[[ 85   0]
 [  2 113]]
1
7/7 [==============================] - 0s 12ms/step - loss: 0.0012 - acc: 1.0000
[0.9200000166893005, 0.9900000095367432, 1.0]
AUC-ROC: 1.0, TP: 110, FP: 0, FN: 0,TN:90
Confusion Matrix:
[[110   0]
 [  0  90]]
2
7/7 [==============================] - 0s 14ms/step - loss: 0.0012 - acc: 1.0000
[0.9200000166893005, 0.9900000095367432, 1.0, 1.0]
AUC-ROC: 1.0, TP: 96, FP: 0, FN: 0,TN:104
Confusion Matrix:
[[ 96   0]
 [  0 104]]
3
7/7 [==============================] - 0s 10ms/step - loss: 2.9328e-05 - acc: 1.0000
[0.9200000166893005, 0.9900000095367432, 1.0, 1.0, 1.0]
AUC-ROC: 1

In [ ]:
!unzip 'drive/My Drive/Action Movie.zip'


In [ ]:
#disk size=9
import numpy as np
import cv2
import glob
from scipy import io
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from skimage.filters.rank import entropy
from skimage.morphology import disk
VideoPath = "Action Movie/fights/"
Videos = glob.glob(VideoPath + "*.avi")
#uint8
XTrainImages=np.zeros((200,100,180,16),dtype='float16')
Counter=0
for vid in Videos:
    X = []
    cap=cv2.VideoCapture(vid)
    ret, frame = cap.read()
    FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    OldFrame=cv2.resize(FrameGray,(180,100))
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            NewFrame=cv2.resize(FrameGray,(180,100))
            EntropyFilter=entropy(abs(NewFrame-OldFrame), disk(9))
            X.append(OldFrame*EntropyFilter+OldFrame)
            OldFrame=NewFrame
            #plt.imshow(FrameGray,cmap=plt.cm.gray)
        else:
            break
    tt=random.sample(X,16)
    for i in range(len(tt)):
        XTrainImages[Counter,:,:,i]=tt[i]
    Counter=Counter+1

VideoPath = "Action Movie/noFights/"
Videos = glob.glob(VideoPath + "*.mpg")
for vid in Videos:
    X = []
    cap=cv2.VideoCapture(vid)
    ret, frame = cap.read()
    FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    OldFrame=cv2.resize(FrameGray,(180,100))
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            FrameGray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            NewFrame=cv2.resize(FrameGray,(180,100))
            EntropyFilter=entropy(abs(NewFrame-OldFrame), disk(9))

            X.append(OldFrame*EntropyFilter+OldFrame)
            OldFrame=NewFrame
            #plt.imshow(FrameGray,cmap=plt.cm.gray)
        else:
            break
    tt=random.sample(X,16)
    for i in range(len(tt)):
        XTrainImages[Counter,:,:,i]=tt[i]
    Counter=Counter+1

#XTrainImages/=255
## Deep Network



In [ ]:
# Fifth Architecture Global average pooling disk size=9
from keras import layers
from keras.models import Model
from keras import optimizers, losses
import numpy as np
from keras.utils import to_categorical
from keras import optimizers
from keras.losses import categorical_crossentropy
TrainLabels=np.zeros((200,1),dtype='int8')
TrainLabels[100:]=1
TrainLabels= to_categorical(TrainLabels)
x_train = np.reshape(XTrainImages, (len(XTrainImages), 100, 180, 16,1))
input_img = layers.Input(shape=(100, 180, 16,1))  # adapt this if using `channels_first` image data format
#x = layers.Conv3D(1, (5,5,5), activation='relu', padding='same')(input_img)
#x=layers.Reshape((50, 80, 25))(x)
x=layers.Reshape((100, 180, 16))(input_img)
y=layers.GlobalAveragePooling2D()(x)

y = layers.Dense(2, activation='relu')(y)
y=layers.Dropout(0.2)(y)
y = layers.Dense(16, activation='sigmoid')(y)
y=layers.Dropout(0.2)(y)

z = tf.keras.layers.Multiply()([y, x])

x = layers.Conv2D(3, (1,1), activation='elu', padding='same')(z)
# z=layers.Reshape((100, 180, 16,1))(z)
# x = layers.Conv3D(1, (5,5,14), activation='elu', padding='valid')(z)
# x=layers.Reshape((96, 176, 3))(x)

x = layers.Conv2D(10, (5,5), activation='elu', padding='same')(x)
x=layers.MaxPool2D(pool_size=(2,2))(x)
x = layers.BatchNormalization()(x)

y=layers.Flatten()(x)
y = layers.Dense(300, activation='elu')(y)
y=layers.Dropout(0.2)(y)

y = layers.Dense(100, activation='elu')(y)
y=layers.Dropout(0.2)(y)

x = layers.Conv2D(20, (3,3), activation='elu', padding='same')(x)
x=layers.MaxPool2D(pool_size=(2,2))(x)
x = layers.BatchNormalization()(x)
x = layers.Conv2D(30, (3,3), activation='elu', padding='same')(x)
x=layers.MaxPool2D(pool_size=(2,2))(x)
x = layers.BatchNormalization()(x)

x = layers.Flatten()(x)
x=layers.Concatenate()([x,y])
x = layers.Dense(300, activation='elu')(x)
x=layers.Dropout(0.2)(x)
x= layers.Dense(100, activation='elu')(x)
x=layers.Dropout(0.2)(x)
out_layer = layers.Dense(2, activation='softmax')(x)
myModel= Model(input_img, out_layer)
## Five Flod cross validation with RMSprop
from sklearn.model_selection import KFold
import numpy as np
import keras
myModel.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.optimizers.RMSprop(learning_rate=0.0001), metrics=['acc'])


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, confusion_matrix
myModel.compile(loss=keras.losses.categorical_crossentropy, optimizer=tf.optimizers.RMSprop(learning_rate=0.0001), metrics=['acc'])

# Assuming myModel is already defined

X = np.arange(len(TrainLabels))
Test_auc_roc = []
Test_precision = []
Test_recall = []
Test_f1 = []
All_confusion_matrices = []
Test_accuracy=[]
Total_TP=[]
Total_FP=[]
Total_FN=[]
Total_TN=[]
Counter = 0
for i in range(6):
    kf = KFold(n_splits=5, random_state=12345, shuffle=True)
    for train_indexInit, test_indexInit in kf.split(X):
        X_trainInit, X_test = x_train[train_indexInit], x_train[test_indexInit]
        y_trainInit, y_test = TrainLabels[train_indexInit], TrainLabels[test_indexInit]

        #print(test_indexInit)

        history = myModel.fit(X_trainInit, y_trainInit, batch_size=16, epochs=20, verbose=0)

        # Assuming myModel.predict is available for obtaining raw predictions
        y_pred = myModel.predict(X_test)
        test_loss, test_acc =myModel.evaluate(X_test, y_test)
        Test_accuracy.append(test_acc)
        print(Test_accuracy)
         # Confusion matrix
        cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
        All_confusion_matrices.append(cm)
        TP=cm[0,0]
        FP=cm[0,1]
        FN=cm[1,0]
        TN=cm[1,1]
        # Calculate ROC-AUC
        auc_roc = roc_auc_score(y_test, y_pred)
        Test_auc_roc.append(auc_roc)
        Total_TP.append(TP)
        Total_FP.append(FP)
        Total_FN.append(FN)
        Total_TN.append(TN)
        print(f"AUC-ROC: {auc_roc}, TP: {TP}, FP: {FP}, FN: {FN},TN:{TN}")
        print("Confusion Matrix:")
        print(cm)

        print(Counter)
        Counter += 1

print("Average AUC-ROC:", np.mean(Test_auc_roc))
print("Average Total_TP:", np.mean(Total_TP))
print("Average Total_FP:", np.mean(Total_FP))
print("Average Total_FN:", np.mean(Total_FN))
print("Average Total_TN:", np.mean(Total_TN))
Average_confusion_matrix = np.mean(All_confusion_matrices, axis=0)
print("Average Confusion Matrix:")
print(Average_confusion_matrix)
print("Test_accuracy",Test_accuracy)
print("Average Test Accurcay",np.mean(Test_accuracy))
print("SD Test Accuracy",np.std(Test_accuracy))
precision=np.mean(Total_TP)/(np.mean(Total_TP)+np.mean(Total_FP))
recall=np.mean(Total_TP)/(np.mean(Total_TP)+np.mean(Total_FN))
f1score=2*precision*recall/(precision+recall)
print("Average precision:", precision)
print("Average recall:",recall)
print("Average f1score:", f1score)



2/2 [==============================] - 1s 15ms/step - loss: 4.7771e-06 - acc: 1.0000
[1.0]
AUC-ROC: 1.0, TP: 13, FP: 0, FN: 0,TN:27
Confusion Matrix:
[[13  0]
 [ 0 27]]
0
2/2 [==============================] - 0s 17ms/step - loss: 4.8128e-06 - acc: 1.0000
[1.0, 1.0]
AUC-ROC: 1.0, TP: 24, FP: 0, FN: 0,TN:16
Confusion Matrix:
[[24  0]
 [ 0 16]]
1
2/2 [==============================] - 0s 32ms/step - loss: 1.1206e-06 - acc: 1.0000
[1.0, 1.0, 1.0]
AUC-ROC: 1.0, TP: 22, FP: 0, FN: 0,TN:18
Confusion Matrix:
[[22  0]
 [ 0 18]]
2
2/2 [==============================] - 0s 29ms/step - loss: 3.6061e-07 - acc: 1.0000
[1.0, 1.0, 1.0, 1.0]
AUC-ROC: 1.0, TP: 24, FP: 0, FN: 0,TN:16
Confusion Matrix:
[[24  0]
 [ 0 16]]
3
2/2 [==============================] - 0s 25ms/step - loss: 1.4901e-08 - acc: 1.0000
[1.0, 1.0, 1.0, 1.0, 1.0]
AUC-ROC: 1.0, TP: 17, FP: 0, FN: 0,TN:23
Confusion Matrix:
[[17  0]
 [ 0 23]]
4
2/2 [==============================] - 0s 16ms/step - loss: 2.3432e-05 - acc: 1.0000
[1.0, 1.0,